# 🌟 Advent of Code 2022

## [Day 15: Beacon Exclusion Zone](https://adventofcode.com/2022/day/15)

In [1]:
import re

def parse_sensors(file):
    input = file.read().strip().splitlines()

    bounds_x = [1e7, -1e7]
    sensors = set()
    beacons = set()
    
    for line in input:
        # find all numbers in line using regex
        sx, sy, bx, by = [int(x) for x in re.findall(r'(-?\d+)', line)]
        distance = manhattan_distance((sx, sy), (bx, by))

        bounds_x = [min(bounds_x[0], sx, bx - distance), max(bounds_x[1], sx, bx + distance)]
        sensors.add((sx, sy, distance))
        beacons.add((bx, by))

    return [sensors, beacons, bounds_x]

def manhattan_distance(point_1, point_2):
    return abs(point_1[0] - point_2[0]) + abs(point_1[1] - point_2[1])

In [2]:
file = open("./inputs/day-15.txt", "r")
sensors, beacons, bounds_x = parse_sensors(file)

### Challenge 1

In [3]:
Y_POS = 2000000

# find out if a beacon can be located at point (x, y). return True if it can, False if it can't
def valid_beacon_position(point): 
    for sx, sy, distance in sensors:
        # if `point` is in range of the sensor (distance from sensor to point is less or equal than
        # the distance between the sensor and its nearest beacon) there can't be a beacon at `point`.
        if manhattan_distance((sx, sy), point) <= distance:
            return False

    return True

impossible_beacon_position_count = 0
for x in range(bounds_x[0], bounds_x[1] + 1):
    point = (x, Y_POS)

    if (not valid_beacon_position(point) and point not in beacons):
        impossible_beacon_position_count += 1

print(f"In the row 'y = 2000000' there are '{impossible_beacon_position_count}' positions that cannot contain a beacon.")

In the row 'y = 2000000' there are '4951427' positions that cannot contain a beacon.


### Challenge 2

According to the description there is only one possible location for the distress beacon.

This means it has to be right outside the range of a sensor **and** within the `x` / `y` coordinates of `0` and `4,000,000`.

In [4]:
# solution adapted from: https://github.com/jonathanpaulson/AdventOfCode/blob/master/2022/15.py

MIN_POS = 0
MAX_POS = 4_000_000

# check for valid beacon position around edge of a sensors 'range'
def search_edge_for_beacon_position(sensor):
    sensor_x, sensor_y, distance = sensor

    # check all points that are `distance + 1` away from the sensor (around the edge of the diamond shape)
    for edge_x in range(distance + 2):
        edge_y = (distance + 1) - edge_x

        for dir_x, dir_y in [(-1,-1),(-1,1),(1,-1),(1,1)]:
            x = sensor_x + (edge_x * dir_x)
            y = sensor_y + (edge_y * dir_y)
            if MIN_POS <= x <= MAX_POS and MIN_POS <= y <= MAX_POS:
                if valid_beacon_position((x, y)):
                    return (x, y)

distress_beacon = None
for sensor in sensors:
    result = search_edge_for_beacon_position(sensor)
    if result:
        distress_beacon = result
        break

tuning_frequency = distress_beacon[0] * 4_000_000 + distress_beacon[1]

print(f"The only possible position for the distress beacon is '{distress_beacon}', with its tuning frequency being '{tuning_frequency}'.")

The only possible position for the distress beacon is '(3257428, 2573243)', with its tuning frequency being '13029714573243'.
